# CDIP Web Scraper

In [1]:
pip install netCDF4

Note: you may need to restart the kernel to use updated packages.


### Import packages

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
from matplotlib import gridspec
import netCDF4
from bs4 import BeautifulSoup
import requests

### Constants

In [7]:
# TODO: load station number and yeardate based on smartfin ride date input
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../smartfin_module')

from smartfin_ride_api import Ride

ride = Ride()
ride.add_rides(ride_ids=['15692'])
dfs = ride.get_rides(ride_ids=['15692'])
times = ride.get_ride_timeframes(ride_ids=['15692'])

fetching ride from: https://surf.smartfin.org/ride/15692
fetching ocean data from: https://surf.smartfin.org/media/201811/google_105349665704999793400_0006667E229D_181109191556_Ocean.CSV
fetching motion data from: https://surf.smartfin.org/media/201811/google_105349665704999793400_0006667E229D_181109191556_Motion.CSV
ride already in here
ride already in here


../smartfin_module\smartfin_ride_api.py:463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  odf['Time'] = [time / 1000 for time in odf['Time']]


In [13]:
times

{'15692': ['09/11/2018 19:16:03', '09/11/2018 20:38:15']}

## Get nearest buoy function

In [8]:




def get_nearest_buoy(latitude, longitude):
    
    # get all active buoys with archived data
    # CDIP active buoys URL
    url="http://cdip.ucsd.edu/m/deployment/station_view/?mode=active"

    # Make a GET request to fetch the raw HTML content
    html_content = requests.get(url).text

    # Parse the html content
    soup = BeautifulSoup(html_content, "lxml")
    table = soup.find("table")
    table_data = table.tbody.find_all("tr")  # contains 2 rows
    stns = []
    for node in table_data:
        try:
            stn = node.findAll('td', text=True)[0]
            stns.append(stn.text.strip(' '))
        except:
            continue
            
    stns.reverse()
    
    # intialize the lowest distance to be some rediculously big number
    lowest_distance = 1000000000
    stn = -1
    longitude /= 100000
    latitude /= 100000
    
    # iterate through 0-450 (station numbers are from 28-433 with gaps in between)
    for i in stns:
        
        # format i into a 3 digit string
        i = str(i)
        if len(i) == 1:
            i = '00' + i
        elif len(i) == 2:
            i = '0' + i
        
        # see if there is a station with the current iteration number
        try:
            data_url = 'http://thredds.cdip.ucsd.edu/thredds/dodsC/cdip/archive/' + i + 'p1/' + i+ 'p1_historic.nc'
            nc = netCDF4.Dataset(data_url)

            # get latitude and longitude of current station
            nc_latitude = nc.variables['metaStationLatitude'][:]
            nc_longitude = nc.variables['metaStationLongitude'][:]

            print(i)
            
#             # if the current station distance is lower than the lowest distance so far, save it
#             curr_distance = abs(nc_latitude - latitude) + abs(nc_longitude - longitude)
#             if curr_distance < lowest_distance:
#                 lowest_distance = curr_distance
#                 stn = i
#                 print('lower: ', i)
#                 print(curr_distance)
#             else: continue

        except OSError as err:
            print("Unexpected error: ", err)
            continue
    
    if stn == -1:
        print('no station found error')
    return stn
            

stn = get_nearest_buoy(1, 1)            

Unexpected error:  [Errno -90] NetCDF: file not found: b'http://thredds.cdip.ucsd.edu/thredds/dodsC/cdip/archive/248p1/248p1_historic.nc'
247
Unexpected error:  [Errno -90] NetCDF: file not found: b'http://thredds.cdip.ucsd.edu/thredds/dodsC/cdip/archive/245p1/245p1_historic.nc'
244
243
240
239
238
236
233
230
226
225
224
222
221
220
217
215
214
213
209
205
204
203
201
200
198
197
196
194
192
191
189
188
187
185
181
179
433
430
171
168
166
163
162
160
158
157
155
154
Unexpected error:  [Errno -90] NetCDF: file not found: b'http://thredds.cdip.ucsd.edu/thredds/dodsC/cdip/archive/153p1/153p1_historic.nc'
150
147
144
143
142
139
134
132
121
106
100
098
094
092
076
071
067
045
036
029
028
no station found error


In [ ]:
stn

In [ ]:
longitude

## (DONT USE) get latitude and longitude of all buoys via webscrape to avoid too many netCDF requests

first way was better and it seems like the amount of netCDF requests is fine

In [ ]:
import re

# # get all active buoys with archived data
# # CDIP active buoys URL
# url="http://cdip.ucsd.edu/m/deployment/station_view/?mode=active"

# # Make a GET request to fetch the raw HTML content
# html_content = requests.get(url).text

# # Parse the html content
# soup = BeautifulSoup(html_content, "lxml")
# table = soup.find("table")
# table_data = table.tbody.find_all("tr")  # contains 2 rows
# stns = []
# for node in table_data:
#     try:
#         stn = node.findAll('td', text=True)[0]
#         stns.append(stn.text.strip(' '))
#     except:
#         continue
        
# # format i into a 3 digit string
# for i in range(len(stns)):
#     stn = str(stns[i])
#     if len(stn) == 1:
#         stn = '00' + stn
#     elif len(stn) == 2:
#         stn = '0' + stn
#     stns[i] = stn
   

# for stn in stns:
#     stn_url = f'https://cdip.ucsd.edu/m/products/?stn={stn}p1'
#     print(stn_url)
#     station_content_html = requests.get(stn_url).text
#     stn_soup = BeautifulSoup(station_content_html, 'html')
#     strongs = stn_soup.find_all('strong', limit=9)
#     latitude = strongs[7].text
#     latitude = re.sub("[^0-9.-]", "", latitude)

#     longitude = strongs[8].text
#     longitude = re.sub("[^0-9.-]", "", longitude)
    
#     print(latitude)
#     print(longitude)

## find active buoy numbers function used above

In [ ]:
# CDIP Archived Dataset URL
from bs4 import BeautifulSoup
import requests

url="http://cdip.ucsd.edu/m/deployment/station_view/?mode=active"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse the html content
soup = BeautifulSoup(html_content, "lxml")
table = soup.find("table")
table_data = table.tbody.find_all("tr")  # contains 2 rows
nums = []
for node in table_data:
    try:
        num = node.findAll('td', text=True)[0]
        nums.append(num.text.strip(' '))
    except:
        continue
        
nums

### Read vars from CDIP THREDDS Server Dataset

In [19]:
# netCDF data object fetched from CDIP API
data_url = 'http://thredds.cdip.ucsd.edu/thredds/dodsC/cdip/archive/' + '124' + 'p1/' + '124'+ 'p1_historic.nc'

nc = netCDF4.Dataset(data_url)

# UNIX based time from 1991-yeardate in 30 minute increments
ncTime = nc.variables['sstTime'][:]
timeall = [datetime.datetime.fromtimestamp(t) for t in ncTime]
waveTime = nc.variables['waveTime'][:]

print(len(waveTime))

# wave heights
Hs = nc.variables['waveHs']
print("hs length: ", len(Hs))
print("ncTime length: ", len(ncTime))

print(ncTime)
print(waveTime)

14732
hs length:  14732
ncTime length:  14732
[1065203849 1065205649 1065207449 ... 1091715978 1091717778 1091719579]
[1065202109 1065203909 1065205709 ... 1091714238 1091716038 1091717839]


In [20]:
# Find nearest value in ncTime array to inputted UNIX Timestamp
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    print('index:',idx)
    return array[idx]

### Convert human time to UNIX time

In [21]:
# Convert human-formatted date to UNIX timestamp
def getUnixTimestamp(humanTime,dateFormat):
    unixTimestamp = int(time.mktime(datetime.datetime.strptime(humanTime, dateFormat).timetuple()))
    print('humanTime: ', humanTime)
    print('UNIX Timestamp: ', unixTimestamp)
    return unixTimestamp

## Time Index Values

Find the UNIX values that correspond to start (Day 1) and end (Day 28) dates for each month. Then find the array index numbers for each UNIX value within the 'ncTime' arrays.

In [26]:
# # Create array of lists of Hs data for each month 

# timeindex_start = []
# timeindex_end = []
# monthcount = 0
start_date = times['15692'][0]
end_date = times['15692'][1]

unixstart = getUnixTimestamp(start_date, "%d/%m/%Y %H:%M:%S")
nearest_date = find_nearest(waveTime, unixstart)  # Find the closest unix timestamp
start_index = np.where(waveTime==nearest_date)[0][0]  # Grab the index number of found date

unixend = getUnixTimestamp(end_date,"%d/%m/%Y %H:%M:%S")
future_date = find_nearest(waveTime, unixend)  # Find the closest unix timestamp
end_index = np.where(waveTime==future_date)[0][0]  # Grab the index number of found date    

# timeindex_start.append(start_index) # Append 'month start date' and 'month end date' index numbers for each month to corresponding array
# timeindex_end.append(end_index)
# 1536718563
# 1536723495
# 309100

humanTime:  09/11/2018 19:16:03
UNIX Timestamp:  1541819763
index: 14731
humanTime:  09/11/2018 20:38:15
UNIX Timestamp:  1541824695
index: 14731


In [27]:
start_index -= 14
end_index -= 14

print(start_index)
print(end_index)

14717
14717


### Calculate mean wave heights of each month

Create array of month-long chunks of Hs data, to be plotted as a series of boxplots. Use a for-loop to cycle through the Hs variable and define each month-long array using the above-specified time index numbers.

In [24]:
box_data = []
i = 0
print(start_index - end_index)
print('Hs length: ', len(Hs))
print('times length: ', len(ncTime))
# all wave height averages per 30 minute increments over each month
ride_hs = Hs[start_index:end_index]
i = i+1
print(len(ride_hs))
print(ride_hs)
    
# append each dataset to the box data
box_data.append(ride_hs)
    
# calculate means of each month dataset in box_data
means = np.asarray([np.mean(m) for m in box_data]) 

# Round each monthly mean value to 2 decimal points, for plotting
meansround = [round(k,2) for k in means] 
print(means)
print(ride_hs.mean())
print('mean wave height:', meansround)
ride_hs.data

0
Hs length:  14732
times length:  14732
0
[]
[nan]
nan
mean wave height: [nan]


C:\Users\evans\anaconda3\lib\site-packages\numpy\ma\core.py:5215: RuntimeWarning: Mean of empty slice.
  result = super(MaskedArray, self).mean(axis=axis,
C:\Users\evans\anaconda3\lib\site-packages\numpy\core\_methods.py:153: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


array([], dtype=float32)

Calculate the number of instances of the variable being incorporated into each month-long average

In [ ]:
monthlengths = []
j = 0

for Hs in range(len(Hs[start_index])):
    
    # how many readings were recorded during that month
    monthlenHs = len(Hs[start_index[j]:start_index[j]])
    j = j+1
    monthlengths.append(monthlenHs)
    
monthlengths

## Plot Wave Time-Series
* Create boxplot graph of month-long datasets
* Each box includes:
    - Median (red line)
    - 25th and 75th percentiles (top and bottom of box)
    - Remaining data within 1.5L above and below (whiskers) the quartiles, where L = length (m) from 25th to 75th percentile
    - Outliers (red crosses) - data beyond whiskers
    - Mean position (green line) and value (green number)
* Adjust colors and labels of graphical display
* Include a second plot of a sample 'legend' boxplot

In [ ]:
# Create overall figure and specify size, and grid to specify positions of subplots
fig = plt.figure(figsize=(12,15)) 
gs = gridspec.GridSpec(2,2,height_ratios=[5,1]) 


# Create a dataset for sample 'legend' boxplot, to go underneath actual boxplot
bp_sample2 = np.random.normal(2.5,0.5,500)


# Create two subplots - actual monthly-averaged data (top) and example 'legend' boxplot (bottom)

# Subplot of monthly-averaged boxplot data
bp = plt.subplot(gs[0,:])
bp_data = bp.boxplot(box_data) # Add 'meanlineprops' to include the above-defined properties
bp.scatter(months,means,marker="_",color='g',linewidths=2.5,s=900) # Overlay monthly means as green lines using 'scatter' function.

# Subplot to show example 'legend' boxplot below actual monthly-averaged boxplot graph
bp2 = plt.subplot(gs[1,1:-1])
bp2_example = bp2.boxplot(bp_sample2,vert=False) # Plot horizontal example boxplot with labels
bp2.scatter(2.3,1,marker="|",color='g',linewidths=2.5,s=400)


# Add values of monthly means as text
for i, txt in enumerate(meansround):
    bp.annotate(txt, (months[i],means[i]),fontsize=12,horizontalalignment='center',verticalalignment='bottom',color='g')

    
# Get positions of Median, Quartiles and Outliers to use in 'legend' text labels 
for line in bp2_example['medians']:
    xm, ym = line.get_xydata()[0] # location of Median line   
for line in bp2_example['boxes']:
    xb, yb = line.get_xydata()[0] # location of Box edges (Quartiles)
for line in bp2_example['whiskers']:
    xw, yw = line.get_xydata()[0] # location of Whisker ends (Outliers)
    

# Add text labels for 'Median', Mean', '25th/75th %iles' and 'Outliers' to subplot2, to create sample 'legend' boxplot
bp2.annotate("Median",[xm,ym-0.3*ym],fontsize=14,color='r')
bp2.annotate("Mean",[2.2,0.65],fontsize=14,color='g')
bp2.annotate("25%ile",[xb-0.01*xb,yb-0.15*yb],fontsize=12)
bp2.annotate("75%ile",[xb+0.2*xb,yb-0.15*yb],fontsize=12)
bp2.annotate("Outliers",[xw+0.38*xw,yw-0.3*yw],fontsize=14,color='r')

 
# Set colors of box aspects for top subplot    
pylab.setp(bp_data['boxes'], color='black')
pylab.setp(bp_data['whiskers'], color='black')
pylab.setp(bp_data['fliers'], color='r')


# Set colors of box aspects for bottom (sample) subplot   
pylab.setp(bp2_example['boxes'], color='black')
pylab.setp(bp2_example['whiskers'], color='black')
pylab.setp(bp2_example['fliers'], color='r')


# Set Titles
plt.suptitle(buoytitle, fontsize=30, y=0.97) # Overall plot title using 'buoytitle' variable
bp.set_title("Significant Wave Height by month for " + year_date, fontsize=20, y=1.01) # Subtitle for top plot
# bp2.set_title("Sample Boxplot", fontsize=16, y=1.02) # Subtitle for bottom plot


# Set axes labels and ticks
bp.set_xticklabels(['ride'],fontsize=12)
bp.set_ylabel('Significant Wave Height, Hs (m)', fontsize=20)
bp.tick_params(axis='y', which='major', labelsize=12, right='off')
bp.tick_params(axis='x', which='major', labelsize=12, top='off')


# Create a second row of x-axis labels for top subplot
newax = bp.twiny()
newax.xaxis.set_ticks_position('bottom')
newax.xaxis.set_label_position('bottom')
newax.spines['bottom'].set_position(('outward',25))
newax.set_xticklabels(monthlengths,fontsize=10)


# Plot horizontal gridlines onto top subplot
bp.grid(axis='y', which='major', color='b', linestyle='-', alpha=0.25)


# Remove tickmarks from bottom subplot
bp2.axes.get_xaxis().set_visible(False)
bp2.axes.get_yaxis().set_visible(False)